In [1]:
import numpy as np
from scipy import sparse
import torch
from torch import nn, optim
from sklearn.metrics import label_ranking_average_precision_score as LRAP

import NeuralNet

In [2]:
torch.cuda.is_available()

False

In [3]:
data = sparse.load_npz("data/data_cleaned.npz")
data

<15539x8993 sparse matrix of type '<class 'numpy.float64'>'
	with 3767303 stored elements in COOrdinate format>

In [4]:
data = data.todense()

X = data[:, :5000]  # features
y = data[:, 5000:5020]  # labels

print(X.shape)
print(y.shape)

(15539, 5000)
(15539, 20)


In [5]:
X = torch.from_numpy(X).float()
X.requires_grad_(True)
print(X.shape)

y = torch.from_numpy(y).float()
print(y.shape)

torch.Size([15539, 5000])
torch.Size([15539, 20])


In [6]:
X

tensor([[0.0846, 0.1386, 0.0943,  ..., 0.0000, 0.0000, 0.0000],
        [0.0507, 0.7623, 0.7544,  ..., 0.0000, 0.0000, 0.0000],
        [0.1015, 0.1386, 0.3772,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0507, 0.3465, 0.1886,  ..., 0.0000, 0.0000, 0.0000],
        [0.0846, 2.3907, 0.8487,  ..., 0.0000, 0.0000, 0.0000],
        [0.0846, 0.2079, 0.2829,  ..., 0.0000, 0.0000, 0.0000]],
       requires_grad=True)

In [7]:
y

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [8]:
net = NeuralNet.Net()
net

Net(
  (l1): Linear(in_features=5000, out_features=1000, bias=True)
  (l2): Linear(in_features=1000, out_features=100, bias=True)
  (l3): Linear(in_features=100, out_features=20, bias=True)
)

In [9]:
criterion = nn.MSELoss()

In [10]:
optimizer = optim.SGD(net.parameters(), lr=0.00001)

In [11]:
for epoch in range(10):
    
    running_loss = 0.0
    for i in np.arange(0, X.shape[0], 1000):
        inp = X[i:i+100, :]
        targets = y[i:i+100, :]
        
        optimizer.zero_grad()
        
        out = net(inp)
        loss = criterion(out, targets)
        loss.backward()
        optimizer.step()
        
        #print(i, loss)
        running_loss += loss.item()
    
    print("Epoch {} Loss: {}".format(epoch, running_loss))
    # validation loss?

Epoch 0 Loss: 7.498230792582035
Epoch 1 Loss: 7.079636454582214
Epoch 2 Loss: 6.71984301507473
Epoch 3 Loss: 6.408346809446812
Epoch 4 Loss: 6.13856253772974
Epoch 5 Loss: 5.901556022465229
Epoch 6 Loss: 5.692268088459969
Epoch 7 Loss: 5.506145879626274
Epoch 8 Loss: 5.33957639336586
Epoch 9 Loss: 5.189771346747875


In [12]:
net

Net(
  (l1): Linear(in_features=5000, out_features=1000, bias=True)
  (l2): Linear(in_features=1000, out_features=100, bias=True)
  (l3): Linear(in_features=100, out_features=20, bias=True)
)

In [13]:
out = net(X)
out = out.detach()

LRAP(y, out)

0.904830487918455

In [14]:
out

tensor([[ 0.0876, -0.0801,  0.0599,  ...,  0.1040, -0.0751, -0.0118],
        [ 0.0650, -0.3998, -0.3408,  ...,  0.2718, -0.0592,  0.1949],
        [ 0.0300, -0.0368, -0.1064,  ...,  0.0763, -0.1050, -0.1031],
        ...,
        [ 0.0812, -0.0725, -0.0490,  ...,  0.1609, -0.0703, -0.0632],
        [ 1.1305, -0.9240, -1.7764,  ...,  0.9205, -0.4070,  0.6921],
        [ 0.2374, -0.1534, -0.0977,  ...,  0.2113,  0.0501,  0.0761]])